In [ ]:
import matplotlib.pyplot as plt
from skimage import io
from skimage import color
from skimage.transform import resize
import math
import numpy as np
import pandas as pd
from PIL import Image
from skimage import data, exposure
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import util
import surf_hog_analysis 

In [ ]:
#!pip install seaborn

In [ ]:
#!pip install imblearn

In [ ]:
#!pip install scikit-learn

---

### Data preparation

In [ ]:
df = pd.read_csv('data/train_complete.csv')

In [ ]:
# isolate only images that have 0 or 1 defect
util.isolate_single_defects(df)

In [ ]:
df.shape

---

# KNN

---

## Use initial train.csv (based on 4 imbalanced classes in train_images) in KNN

In [ ]:
# eliminate class 0 in dataframe
df2 = df.query('ClassId != 0')

In [ ]:
df.groupby('ClassId')['ImageId'].count()

In [ ]:
df.head()

In [ ]:
initial_images =[]

IMG_SIZE = 128
for image_id in df2['ImageId']:
    image = io.imread('data/single_defect_train_images/' + image_id)
    #resize images
    image_resized=cv2.resize(image,(IMG_SIZE,IMG_SIZE))
    #The input data have to be converted from 3 dimensional format to 1 dimensional format
    image_flat = image_resized.reshape(1, 3*IMG_SIZE*IMG_SIZE)
    # Data Normalization
    # Conversion to float
    image_flat=image_flat.astype('float32')
    # Normalization (In the RGB color space the red, green and blue have integer values from 0 to 255)
    image_flat = image_flat/255.0
    initial_images.append([image_id,image_flat[0]])

In [ ]:
initial_images = pd.DataFrame(initial_images , columns = ['ImageId', 'flattened_images'])

In [ ]:
initial_images.head()

In [ ]:
split_initial_images = pd.DataFrame(initial_images['flattened_images'].tolist())
initial_images_complete = pd.concat([initial_images, split_initial_images], axis=1)
initial_images_complete.drop(['flattened_images'], axis=1, inplace=True)

In [ ]:
initial_images_complete.head()

In [ ]:
# add classId to dataframe
initial_images_complete = pd.merge(initial_images_complete, df[['ImageId','ClassId']], on='ImageId')

In [ ]:
initial_images_complete.head()

In [ ]:
#Split in X und Y
X = initial_images_complete.drop(['ClassId','ImageId'], axis =1)
y = initial_images_complete['ClassId']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y, random_state = 42)
print('Training data and target sizes: \n{}, {}'.format(X_train.shape,y_train.shape))
print('Test data and target sizes: \n{}, {}'.format(X_test.shape,y_test.shape))

In [ ]:
test_accuracy = []
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

classifier = KNeighborsClassifier(n_neighbors=3,algorithm='brute')
classifier.fit(X_train_scaled, y_train)


y_pred = classifier.predict(X_test_scaled)
test_accuracy = classifier.score((X_test_scaled), y_test)
print(test_accuracy)

In [ ]:
print(metrics.confusion_matrix(y_test,y_pred))
# Print confusion matrix
sns.heatmap(metrics.confusion_matrix(y_test, y_pred), annot=True, cmap='YlGn');

In [ ]:
print(metrics.classification_report(y_test, y_pred))

---

### Visualization of falsely predicted pictures

---

#### Piece-by-piece visualization

In [ ]:
# extract indices of our `hog_complete` data frame where predictions were incorrect
false_predictions = (y_pred!=y_test)
false_predictions = false_predictions[false_predictions].index.values

In [ ]:
y_pred[(y_pred!=y_test)]

In [ ]:
y_pred_false =y_pred[(y_pred!=y_test)]

predictions = pd.DataFrame(y_pred_false, index=false_predictions,columns=['ClassId_predicted'])
predictions

In [ ]:
# extract all rows from `hog_complete` where the predcition was incorrect
false_predicted_images = hog_complete.join(predictions).loc[false_predictions][['ImageId','ClassId', 'ClassId_predicted']]
# add additional information needed to find the correponding pictures
false_predicted_images = false_predicted_images.merge(df[['FilePath','ImageId']], on = 'ImageId')


In [ ]:
false_predicted_images

In [ ]:
number_images = 5

random_index = np.round(np.random.rand(number_images) * len(false_predicted_images.ImageId)) + 1
#print(random_index)

for i in range(number_images):
    #random_index = 'random_index_' + str(i+1)
    #print(random_index)
    file_path_to_image = false_predicted_images['FilePath'][random_index[i]]
    class_id = false_predicted_images['ClassId'][random_index[i]]
    image_id = false_predicted_images['ImageId'][random_index[i]]
    class_id_pred = int(false_predicted_images['ClassId_predicted'][random_index[i]])
    
    img = io.imread(file_path_to_image)
    plt.figure(figsize=(18, 10))
    ax = plt.subplot(number_images, 1, i + 1)
    plt.imshow(img)
    plt.title(f'Image ID: {image_id} | True ClassId: {class_id} | Predicted ClassId: {class_id_pred}', fontsize=16);
    plt.axis("off")